<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/CNN_from_scratch_with_tf_keras_layer's_subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are on CoLab with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n...please load your drive...")
  COLAB = False
def time_fmt(t:float = 231.1829)->float:
  h = int(t / (60 * 60)/ 60)
  m = int(t % (60 * 60)/ 60)
  s = int(t % 60)
  return f"{h}: {m:>03}: {s:>05.2f}"
print(f"....time testing...time testing...time testing...\ntime elapse: {time_fmt()}")

Mounted at /content/drive
You are on CoLab with tensorflow version: 2.4.1
....time testing...time testing...time testing...
time elapse: 0: 003: 51.00


In [ ]:
#We are going to train a cnn model on mnist dataset from scratch:


In [ ]:
import time, os
import tensorflow as tf
import numpy as np


In [ ]:
#Get and reshape the mnist dataset:

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
print(f"x_train_shape: {x_train.shape}, x_test_shape: {x_test.shape}\ny_train_shape: {y_train.shape}, y_test_shape: {y_test.shape}")

x_train_shape: (60000, 28, 28), x_test_shape: (10000, 28, 28)
y_train_shape: (60000,), y_test_shape: (10000,)


In [ ]:
#Reshape the images and adding the channel dimension:
x_train, x_test = x_train.reshape((-1, 28,28,1)), x_test.reshape((-1,28,28,1))

In [ ]:
#Convert into numpy and scale into [0,1]

In [ ]:
x_train, x_test = x_train.astype(np.float32)/255.0, x_test.astype(np.float32)/255.0

In [ ]:
print(f"x_train_shape: {x_train.shape}, x_test_shape: {x_test.shape}")

x_train_shape: (60000, 28, 28, 1), x_test_shape: (10000, 28, 28, 1)


In [ ]:
#Building the classes for the labels: (10 classes)

In [ ]:
y_train, y_test = tf.keras.utils.to_categorical(y_train, num_classes = 10), tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [ ]:
#Convert to tensorflow datatypes:

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
BUFFER = 1024
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(BUFFER).batch(batch_size = BATCH_SIZE, drop_remainder = True)
validation_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
validation_data = validation_data.shuffle(BUFFER).batch(batch_size = BATCH_SIZE, drop_remainder = True)
x_train_batch_sample, y_train_batch_sample = next(iter(train_data))
print(f"x_train_batch_sample_shape: {x_train_batch_sample.shape}, y_train_batch_sample_shape: {y_train_batch_sample.shape}")

x_train_batch_sample_shape: (64, 28, 28, 1), y_train_batch_sample_shape: (64, 10)


In [ ]:
#We now build our model using layer subclassing for convolution block:

In [ ]:
class ConvBlock(tf.keras.layers.Layer):
  def __init__(self, num_filters, kernel = 3, *args, **kwargs):
    super(ConvBlock, self).__init__(*args, **kwargs)
    self.conv = tf.keras.layers.Conv2D(filters = num_filters, kernel_size = kernel, padding = 'same', activation = 'relu')
    self.bn = tf.keras.layers.BatchNormalization()
  
  def call(self, inputs, training = False):
    x = self.conv(inputs)
    x = self.bn(x)
    return x



In [ ]:
#We can build our model as follow:
model = tf.keras.models.Sequential(
    [ConvBlock(64),
     ConvBlock(128),
     ConvBlock(256),
     ConvBlock(512),
     ConvBlock(128),
     ConvBlock(64),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(units = 1024, activation = 'relu'),
     tf.keras.layers.Dropout(rate = 0.5),
     tf.keras.layers.Dense(units = 512, activation = 'relu'),
     tf.keras.layers.Dense(units = 10, activation = 'softmax')]
)

In [ ]:
#The training loop from scratch:

In [ ]:
loss_obj = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
train_metric = tf.keras.metrics.CategoricalAccuracy()
eval_metric = tf.keras.metrics.CategoricalAccuracy()


In [ ]:
tic = time.time()
for epoch in range(EPOCHS):
  print(f"The train starts at epoch: {epoch + 1}")

  for (step, (x_train_batch, y_train_batch)) in enumerate(train_data):
    with tf.GradientTape() as tape:
      preds = model(x_train_batch, training = True)
      train_loss = loss_obj(y_train_batch,preds)
    grads = tape.gradient(train_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_metric.update_state(y_train_batch, preds)
    train_acc = train_metric.result()
    train_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1} : train accuracy is: {float(train_acc):.4f}")
      print(f"at batch number: {step} : training loss is: {float(train_loss):.4f}")
  
  for (step, (x_eval_batch, y_eval_batch)) in enumerate(validation_data):
    preds = model(x_eval_batch, training = False)
    eval_loss = loss_obj(y_eval_batch, preds)
    eval_metric.update_state(y_eval_batch, preds)
    eval_acc = eval_metric.result()
    eval_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch : {epoch + 1}: validation accuracy is: {float(eval_acc):.4f}")
      print(f"at batch number {step}: validation loss is: {float(eval_loss):.4f}")
toc = time.time()
print(f"\ntotal time elapsed in seconds: {time_fmt(toc - tic)}")

The train starts at epoch: 1
epoch: 1 : train accuracy is: 0.1719
at batch number: 0 : training loss is: 69.7695
epoch: 1 : train accuracy is: 0.9531
at batch number: 200 : training loss is: 1.4176
epoch: 1 : train accuracy is: 0.8906
at batch number: 400 : training loss is: 1.3086
epoch: 1 : train accuracy is: 0.9062
at batch number: 600 : training loss is: 1.1646
epoch: 1 : train accuracy is: 0.9531
at batch number: 800 : training loss is: 0.2233
epoch : 1: validation accuracy is: 0.9688
at batch number 0: validation loss is: 0.1420
The train starts at epoch: 2
epoch: 2 : train accuracy is: 0.9531
at batch number: 0 : training loss is: 0.2648
epoch: 2 : train accuracy is: 0.9531
at batch number: 200 : training loss is: 0.2501
epoch: 2 : train accuracy is: 1.0000
at batch number: 400 : training loss is: 0.0091
epoch: 2 : train accuracy is: 0.9688
at batch number: 600 : training loss is: 0.1335
epoch: 2 : train accuracy is: 1.0000
at batch number: 800 : training loss is: 0.0066
epoch :